In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import datetime

from summer.utils import ref_times_to_dti

from autumn.tools.plots.utils import REF_DATE
from autumn.tools.runs import ManagedRun
from autumn.tools.runs.calibration.utils import get_posteriors
from autumn.tools.utils.pandas import pdfilt
from autumn.settings.region import Region

## Specify the runs of interest

In [ ]:
run_id = "covid_19/hanoi/1643227586/0c2e8ae"

In [ ]:
mr = ManagedRun(run_id)

In [ ]:
pbi = mr.powerbi.get_db()
targets = pbi.get_targets()
results = pbi.get_uncertainty()
mcmc_params = mr.calibration.get_mcmc_params()

In [ ]:
results.columns

In [ ]:
mcmc_params.columns

## Dictionaries for lookup and scenario definition

In [ ]:
# calibrated parameters name lookup
param_lookup = {
    "contact_rate": "infection risk per contact",
    "infectious_seed": "starting infectious population",
    "sojourn.compartment_periods_calculated.active.total_period": "active disease duration",
    "sojourn.compartment_periods_calculated.exposed.total_period": "incubation period",
    "testing_to_detection.assumed_cdr_parameter": "CDR at 1/10,000 daily testing rate",
    "mobility.microdistancing.behaviour.parameters.max_effect": "max effect of microdistancing behaviors",
    "clinical_stratification.props.hospital.multiplier": "relative multiplier of rate of admission to hospitals",
    "infection_fatality.multiplier": "relative multiplier to fatality rate",
    "clinical_stratification.icu_prop": "proportion of ICU admission from hospitalised patients"
}

# calibrated parameters unit lookup
unit_lookup = {
    "contact_rate": "probability",
    "infectious_seed": "persons",
    "sojourn.compartment_periods_calculated.active.total_period": "days",
    "sojourn.compartment_periods_calculated.exposed.total_period": "days",
    "testing_to_detection.assumed_cdr_parameter": "proportion",
    "mobility.microdistancing.behaviour.parameters.max_effect": "proportion",
    "clinical_stratification.props.hospital.multiplier": "ratio",
    "infection_fatality.multiplier": "ratio",
    "clinical_stratification.icu_prop": "proportion"
}

# derived output figure title lookup
title_lookup = {
    "accum_deaths": "Cumulative number of COVID-19-specific fatalities",
    "cdr": "Case Detection Rate (in ratio)",
    "hospital_occupancy": "Total actually occupied hospital beds",
    "icu_occupancy": "Total actuall occupied beds for critical patients",
    "incidence": "Daily actual number of COVID-19 cases",
    "infection_deaths": "Daily number of COVID-19-specific fatalities",
    "notifications": "Daily number of registered cases"
}

In [ ]:
# number of scenarios and their names(including baseline)
#scenario_names = ("Baseline", "Scenario 1", "Scenario 2", "Scenario 3", "Scenario 4", "Scenario 5", "Scenario 6", "Scenario 7", "Scenario 8")
scenario_names = {0: "Baseline scenario",
                  1: "Scenario 1", 
                  2: "Scenario 2",
                  3: "Scenario 3",
                  4: "Scenario 4",
                  5: "Scenario 5",
                  6: "Scenario 1 - Late Booster",
                  7: "Scenario 2 - Late Booster",
                  8: "Scenario 3 - Late Booster"}
n_scenarios = len(scenario_names)

## Display parameter posterior histograms

In [ ]:
plt.style.use("ggplot")

fig, axes = plt.subplots(3, 3, figsize=(18, 18), sharey="all")
for i_ax, axis in enumerate(axes.reshape(-1)):
    param = mcmc_params.columns[i_ax]
    axis.hist(mcmc_params[param])
    axis.set_title(param_lookup[param])
    axis.set_xlabel(unit_lookup[param])
    if i_ax % 3 == 0:
        axis.set_ylabel("runs")   
fig.suptitle("parameter posterior histograms", fontsize=15, y=1)
fig.tight_layout()

In [ ]:
# display all derived outputs
results.columns.levels[0]

In [ ]:
# calibration fits for notifications, ICU and deaths
# dark shading - 50% confidence interval, light shading - 95% confidence interval
fig, axes = plt.subplots(2, 2, figsize = (15, 10), sharex="all")
plot_left_date = datetime.date(2021, 5, 25)
plot_right_date = datetime.date(2022 , 1, 25)
colour = (0.2, 0.2, 0.8)
outputs = ("notifications", "icu_occupancy", "infection_deaths")
fig_names = ("Number of daily confirmed cases", "ICU occupancy", "Number of daily confirmed fatalities")
for i_ax, axis in enumerate(axes.reshape(-1)):
    if i_ax < len(outputs):
        results_df = results[(outputs[i_ax], 0)][plot_left_date:plot_right_date]
        indices = results_df.index
        axis.scatter(targets.index, targets[outputs[i_ax]], color="k", alpha=1., s=10)
        axis.plot(indices, results_df[0.5], linewidth=1)
        axis.fill_between(indices, results_df[0.025], results_df[0.975], color=colour, alpha=0.5)
        axis.fill_between(indices, results_df[0.25], results_df[0.75], color=colour, alpha=0.7)
        axis.set_title(fig_names[i_ax], fontsize=20)
    axis.tick_params(axis="x", labelrotation=45)
    axis.set_xlim(left=plot_left_date)
fig.delaxes(axes[1][1])
        
fig.suptitle("Hanoi model calibration for notifications, ICU occupancy and deaths until 25th Jan 2022", fontsize=23)
fig.savefig("./Hanoi_calibration_012722.png")

In [ ]:
def plot_all_scenarios_single_output(output):
    plot_left_date = datetime.date(2021, 5, 5)
    plot_right_date = datetime.date(2022, 6, 30)
    start_date_scenarios = datetime.date(2022, 2, 1)
    fig, axis = plt.subplots(1, 1, figsize=(15, 10))
    plt.axvline(x=start_date_scenarios, linestyle="--", color="k")
    plt.text(start_date_scenarios, 0.95, '2022-02-01 ', transform=axis.get_xaxis_transform(), ha="right", va="top", fontsize=12)
    for scenario in scenario_names.keys():
        if scenario == 0:
            results_df = results[(output, scenario)][plot_left_date:start_date_scenarios]
            indices = results_df.index
            axis.plot(indices, results_df[0.5], linewidth=2)
        else:
            results_df = results[(output, scenario)][plot_left_date:plot_right_date]
            indices = results_df.index
            axis.plot(indices, results_df[0.5], label = scenario_names[scenario], linewidth=2)
    
    axis.tick_params(axis="x", labelrotation=45)
    axis.set_xlim(left=plot_left_date)
    axis.set_title(f"{title_lookup[output]} for all scenarios", fontsize=16)
    plt.legend(fontsize=10)
    # fig.savefig(f"./Hanoi_{output}_all_scenarios.png")

In [ ]:
for output in ("notifications", "icu_occupancy", "infection_deaths"):
    plot_all_scenarios_single_output(output)

In [ ]:
def plot_all_outputs_all_scenarios():
    plot_left_date = datetime.date(2021, 5, 5)
    plot_right_date = datetime.date(2022, 6, 30)
    start_date_scenarios = datetime.date(2022, 2, 1)
    
    outputs_to_plot = ("notifications", "hospital_occupancy", "icu_occupancy", "infection_deaths")
    
    fig, axes = plt.subplots(2, 2, figsize = (21, 14), sharex="all")
    fig.set_facecolor("w")
    
    for i_ax, axis in enumerate(axes.reshape(-1)):
        axis.axvline(x=start_date_scenarios, linestyle="--", color="k")
        plt.text(start_date_scenarios, 0.95, '2022-02-01 ', transform=axis.get_xaxis_transform(), ha="right", va="top", fontsize=12)
        for scenario in scenario_names.keys():
            if scenario == 0:
                # plot baseline before the first date of scenarios
                results_df = results[(outputs_to_plot[i_ax], scenario)][plot_left_date:start_date_scenarios]
                indices = results_df.index
                axis.plot(indices, results_df[0.5], linewidth=2)
            else:
                results_df = results[(outputs_to_plot[i_ax], scenario)][plot_left_date:plot_right_date]
                indices = results_df.index
                axis.plot(indices, results_df[0.5], label = scenario_names[scenario], linewidth=2)
        
        axis.tick_params(axis="x", labelrotation=45)
        axis.set_xlim(left=plot_left_date)
        axis.set_title(title_lookup[outputs_to_plot[i_ax]], fontsize=20)
        axis.legend()
    fig.suptitle("Hanoi modelling results of all scenarios, for all outputs", fontsize=25)
    # fig.savefig("./Hanoi_all_scens_all_outputs.png")
        
plot_all_outputs_all_scenarios()

In [ ]:
def plot_all_outputs_chosen_scenarios(scenarios_to_plot, name_to_save):
    plot_left_date = datetime.date(2021, 5, 5)
    plot_right_date = datetime.date(2022, 6, 30)
    start_date_scenarios = datetime.date(2022, 2, 1)
    
    outputs_to_plot = ("notifications", "hospital_occupancy", "icu_occupancy",
                       "infection_deaths")
    
    fig, axes = plt.subplots(2, 2, figsize = (21, 14), sharex="all")

    for i_ax, axis in enumerate(axes.reshape(-1)):
        axis.axvline(x=start_date_scenarios, linestyle="--", color="k")
        plt.text(start_date_scenarios, 0.95, '2022-02-01 ', transform=axis.get_xaxis_transform(), ha="right", va="top", fontsize=12)
        # plot baseline before the first date of scenarios
        results_df = results[(outputs_to_plot[i_ax], 0)][plot_left_date:start_date_scenarios]
        indices = results_df.index
        axis.plot(indices, results_df[0.5], linewidth=2)
        for scenario in scenarios_to_plot:
            results_df = results[(outputs_to_plot[i_ax], scenario)][plot_left_date:plot_right_date]
            indices = results_df.index
            axis.plot(indices, results_df[0.5], label = scenario_names[scenario], linewidth=2)
        
        axis.tick_params(axis="x", labelrotation=45)
        axis.set_xlim(left=plot_left_date)
        axis.set_title(title_lookup[outputs_to_plot[i_ax]], fontsize=20)
        axis.legend()
    
    fig.suptitle(name_to_save, fontsize=25)
    # fig.savefig(f"./Hanoi_scens_{scenarios_to_plot}_all_outputs.png")

In [ ]:
def plot_icu_deaths_chosen_scenarios(scenarios_to_plot, name_to_save):
    plot_left_date = datetime.date(2021, 5, 5)
    plot_right_date = datetime.date(2022, 6, 30)
    start_date_scenarios = datetime.date(2022, 2, 1)
    
    outputs_to_plot = ("icu_occupancy", "infection_deaths")
    
    fig, axes = plt.subplots(1, 2, figsize = (21, 7), sharex="all")

    for i_ax, axis in enumerate(axes.reshape(-1)):
        axis.axvline(x=start_date_scenarios, linestyle="--", color="k")
        plt.text(start_date_scenarios, 0.95, '2022-02-01 ', transform=axis.get_xaxis_transform(), ha="right", va="top", fontsize=12)
        # plot baseline before the first date of scenarios
        results_df = results[(outputs_to_plot[i_ax], 0)][plot_left_date:start_date_scenarios]
        indices = results_df.index
        axis.plot(indices, results_df[0.5], linewidth=2)
        for scenario in scenarios_to_plot:
            results_df = results[(outputs_to_plot[i_ax], scenario)][plot_left_date:plot_right_date]
            indices = results_df.index
            axis.plot(indices, results_df[0.5], label = scenario_names[scenario], linewidth=2)
        
        axis.tick_params(axis="x", labelrotation=45)
        axis.set_xlim(left=plot_left_date)
        axis.set_title(title_lookup[outputs_to_plot[i_ax]], fontsize=15)
        axis.legend()
    
    fig.suptitle(name_to_save, fontsize=20)
    fig.savefig(f"./Hanoi_icu_deaths_scens_{scenarios_to_plot}_all_outputs.png")

In [ ]:
plot_all_outputs_chosen_scenarios([1, 2, 3, 4, 5], 
                                  "Hanoi modelling results for all scenarios with early booster dose")

In [ ]:
plot_all_outputs_chosen_scenarios([1, 2, 3, 6, 7, 8], 
                                  "Difference in modelling outcomes of Hanoi when booster dose are administered at different intervals")

In [ ]:
plot_icu_deaths_chosen_scenarios([1, 2, 3, 4, 5], 
                                  "Hanoi modelling results for all scenarios with early booster dose")

In [ ]:
plot_icu_deaths_chosen_scenarios([1, 2, 3, 6, 7, 8], 
                                  "Difference in modelling outcomes of Hanoi when booster dose are administered at different intervals")

## Print all necessary epidemiological information of a scenario

In [ ]:
# note that scenarios are numbered from 0 for baseline, and 1, 2, 3, etc. onwards
start_date_scenarios = datetime.date(2022, 2, 1)
end_date_scenarios = datetime.date(2022, 6, 30)
def print_epi_info(scenario):
    print("From {} to {}, for {}:".format(start_date_scenarios, end_date_scenarios, scenario_names[scenario]))
    print("\b The cumulative number of fatalities of scenario {} within this time frame is {}.".format(scenario, 
           round(results[("accum_deaths", scenario)][0.5][end_date_scenarios] - results[("accum_deaths", scenario)][0.5][start_date_scenarios])))
    print("\b The necessary number of hospital beds needed to fulfill the occupancy is {}."
          .format(round(max(results[("hospital_occupancy", scenario)][0.5][start_date_scenarios:end_date_scenarios]))))
    print("\b The necessary number of ICU beds needed to fulfill the occupancy is {}."
          .format(round(max(results[("icu_occupancy", scenario)][0.5][start_date_scenarios:end_date_scenarios]))))
    print("\b The maximum number of incidence cases (including asymptomatic and undetected cases) is {}."
          .format(round(max(results[("incidence", scenario)][0.5][start_date_scenarios:end_date_scenarios]))))
    print("\b The maximum number of daily COVID-19-specific fatalities is {}."
          .format(round(max(results[("infection_deaths", scenario)][0.5][start_date_scenarios:end_date_scenarios]))))
    print("\b The maximum number of daily COVID-19 registered cases is {}."
          .format(round(max(results[("notifications", scenario)][0.5][start_date_scenarios:end_date_scenarios]))))
    print("\b The total number of daily COVID-19 registered cases is {}."
          .format(round(sum(results[("notifications", scenario)][0.5][start_date_scenarios:end_date_scenarios]))))

In [ ]:
for i in scenario_names.keys():
    print_epi_info(i)